In [1]:
# pets don't get arp
# pets get little to no +hit other than boomkin

In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
patch = 2.0

# talents
## marksman
lethal_shots = 5
ranged_weapon_specialization = 5
RWSMod = 1 + ranged_weapon_specialization * 0.01
trueshot_aura = 1
rapid_killing = 2
combat_experience = 2
mortal_shots = 5

## survival
humanoid_monster_slaying = 0

## BM
serpents_swiftness = 5
improved_aoth = 5
animal_handler = 2
frenzy = 5
frenzy_modifier = 0
bestial_discipline = 2
unleashed_fury = 5
focused_fire = 2

# gear
WeaponSpeed = 3
WeaponMinimumDamage = 217
WeaponMaximumDamage = 327
ScopeDamage = 0
AmmoDPSLow = 53 # private server, in tbc is het 46
AmmoDPSHigh = 53 # private server, in tbc is het 47
gear_rap = 38
gear_critrating = 16
gear_haste_rating = 0
quiver_modifier = 0.15
aoth_modifier = 0
gear_agi = 0
gear_hit_rating = 100


# base stats for dwarf hunter
base_str = 66
base_agi = 147
base_int = 76
base_spi = 82
base_mana = 3253
base_critrating = 22.08
base_ap = 3
buffed_agi = (base_agi + gear_agi) * (1 + 0.01 * combat_experience)
rap = 2*(buffed_agi-10) + base_ap + gear_rap + 125 * trueshot_aura
base_hit_rating = 0
current_level = 70
AttackerSkill = 5 * current_level #dit is een synoniem voor player attack rating
NormalMissChance = 0.05
AttackerCrit = (base_critrating + gear_critrating)/22.08/100 + ((base_agi + gear_agi) * 0.025)/100  + lethal_shots * 0.01
human_latency = 0

# stats for enemy mob
imp_faerie_fire = True
faerie_fire = True
curse_of_recklessness = True
sunder_armor_stacks = 5
mob_armor = 7700 - 520 * sunder_armor_stacks -\
                   610 * max(faerie_fire, imp_faerie_fire) -\
                   800 * curse_of_recklessness
mob_level = 73
TargetLevel = mob_level
mob_defense_rating = 5 * mob_level
mob_wears_shield = True
Dazed = 0

# pet base stats
pet_level = 70
pet_base_str = 162
pet_base_agi = 128
pet_base_int = 33
pet_base_spi = 99
pet_base_critrating = 0
pet_base_min_attack = 50
pet_base_max_attack = 70
pet_base_ap = 125 * trueshot_aura + (base_str - 10)*2 + 0.22 * rap
pet_crit = (pet_base_agi/25.6 + ((5*pet_level)-(5*mob_level))*0.04)/100
pet_ap_buffs = 264
petAttackerSkill = 5 * pet_level
frenzy_modifier = 0
pet_family = 'ravager'

if pet_family == 'ravager':
    pet_family_bonus = 0.10

pet_happiness = 'happy'
if pet_happiness == 'happy':
    pet_happiness_bonus = 0.25
elif pet_happiness == 'content':
    pet_happiness_bonus = 0
elif  pet_happiness == 'content':
    pet_happiness_bonus = -0.25

# pet talents
cobra_reflexes = True

# andere bewaarvariabelen
number_of_autoattacks = 0
totaldmg_autoattacks = 0.0

In [4]:
DamageModifiers = 1 

auto_attack_damage_lowend = DamageModifiers*(AmmoDPSLow*WeaponSpeed + RWSMod*((rap/14)*WeaponSpeed + WeaponMinimumDamage) + ScopeDamage) 
auto_attack_damage_highend = DamageModifiers*(AmmoDPSHigh*WeaponSpeed + RWSMod*((rap/14)*WeaponSpeed + WeaponMaximumDamage) + ScopeDamage) 
auto_attack_damage = (auto_attack_damage_lowend + auto_attack_damage_highend) /2
SteadyShotDamage_lowend = DamageModifiers*(150 + (auto_attack_damage_lowend/WeaponSpeed)*2.8 + 0.2*rap + Dazed*175)
SteadyShotDamage_highend = DamageModifiers*(150 + (auto_attack_damage_highend/WeaponSpeed)*2.8 + 0.2*rap + Dazed*175)
SteadyShotDamage = (SteadyShotDamage_lowend + SteadyShotDamage_highend) / 2
print(f'auto_attack_damage_lowend = {auto_attack_damage_lowend}')
print(f'auto_attack_damage_highend = {auto_attack_damage_highend}')
print(f'SteadyShotDamage = {SteadyShotDamage}')
print(f'SteadyShotDamage_lowend = {SteadyShotDamage_lowend}')
print(f'SteadyShotDamage_highend = {SteadyShotDamage_highend}')
# print(f'KillCommandDamage = {KillCommandDamage}')

auto_attack_damage_lowend = 487.173
auto_attack_damage_highend = 602.673
SteadyShotDamage = 747.7708
SteadyShotDamage_lowend = 693.8708
SteadyShotDamage_highend = 801.6708


In [5]:
# hunter attacks can miss, crit, crushing blow, hit. They cannot be parried, dodged

#Hit percentage
HitChance = (gear_hit_rating + base_hit_rating) / 30.7548 / 100 + imp_faerie_fire * 0.03
print(f'HitChance = {HitChance}')

PetHitChance = imp_faerie_fire * 0.03 + animal_handler * 0.02
print(f'PetHitChance = {PetHitChance}')


#Mob armor
ArmorReductionFactor = mob_armor/(mob_armor-22167.5+(467.5*70))
print(f'ArmorReductionFactor = {ArmorReductionFactor}')


#Miss
if TargetLevel < 10:
    MissChance = max(0, NormalMissChance * (TargetLevel / 10) - HitChance)
elif (mob_defense_rating - AttackerSkill) <= 10:
    MissChance = max(0, 0.05 + (TargetLevel * 5 - AttackerSkill) * 0.001 - HitChance)
elif (mob_defense_rating - AttackerSkill) >= 11:
    MissChance = max(0, 0.05 + (TargetLevel * 5 - AttackerSkill) * 0.002 - HitChance)
print(f'MissChance = {MissChance}')

if TargetLevel < 10:
    PetMissChance = max(0, NormalMissChance * (TargetLevel / 10) - PetHitChance)
elif (mob_defense_rating - petAttackerSkill) <= 10:
    PetMissChance = max(0, 0.05 + (TargetLevel * 5 - petAttackerSkill) * 0.001 - PetHitChance)
elif (mob_defense_rating - petAttackerSkill) >= 11:
    PetMissChance = max(0, 0.05 + (TargetLevel * 5 - petAttackerSkill) * 0.002 - PetHitChance)
print(f'PetMissChance = {PetMissChance}')


#Block (only from front)
if mob_wears_shield:
    BlockChance = max(0, min(0.05, 0.05 + (TargetLevel*5 - petAttackerSkill) * 0.001))
else: BlockChance = 0
print(f'BlockChance = {BlockChance}')

if mob_wears_shield:
    PetBlockChance = max(0, min(0.05, 0.05 + (TargetLevel*5 - petAttackerSkill) * 0.001))
else: PetBlockChance = 0
print(f'PetBlockChance = {PetBlockChance}')


#Crit
if (AttackerSkill - mob_defense_rating) < 0:
    CritChance = AttackerCrit + (AttackerSkill - mob_defense_rating) * 0.002
elif (mob_defense_rating - AttackerSkill) >= 0:
    CritChance = AttackerCrit + (AttackerSkill - mob_defense_rating) * 0.004
print(f'CritChance = {CritChance}')

if (petAttackerSkill - mob_defense_rating) < 0:
    PetCritChance = pet_crit + (petAttackerSkill - mob_defense_rating) * 0.002
elif (mob_defense_rating - AttackerSkill) >= 0:
    PetCritChance = pet_crit + (petAttackerSkill - mob_defense_rating) * 0.004
print(f'PetCritChance = {PetCritChance}')

# Crit enhancement factor, double dips on slaying, once for the crit component and once for the dmg component
CritEnhancementFactor = (1 + 0.01 * humanoid_monster_slaying)\
                         *\
                        (2 + 0.06 * mortal_shots*(1 + 0.01 * humanoid_monster_slaying))
print(f'CritEnhancementFactor = {CritEnhancementFactor}')


# melee attacks have more options than ranged, here's the stats for pets:

#Glancing
# glancing gebeurt alleen op white non-crits
if patch < 2.1:
    GlancingChance = 0.10 + max(0, (TargetLevel - pet_level) * 0.1)
else:
    GlancingChance = 10 + mob_defense_rating - petAttackerSkill 
LowEndGlancing  = min(0.91, 1.3 - 0.05*(mob_defense_rating - petAttackerSkill))
HighEndGlancing = max(0.2, min(0.99, 1.2 - 0.03*(mob_defense_rating - petAttackerSkill)))
GlancingReduction =  max(0, (HighEndGlancing + LowEndGlancing) / 2)
print(f'GlancingChance = {GlancingChance}')
print(f'GlancingReduction = {GlancingReduction}')

#Dodge
DodgeChance = 0.05 + (TargetLevel*5 - petAttackerSkill) * 0.001
print(f'DodgeChance = {DodgeChance}')

#Parry (only from front)
if (mob_level - pet_level) < 3:
    ParryChance = 0.05 + 0.005 * (mob_level - pet_level)
elif (mob_level - pet_level) >= 3:
    ParryChance = 0.125 + 0.005 * (mob_level - pet_level)
print(f'ParryChance = {ParryChance}')

print(f'[Miss] if < {round(PetMissChance,5)}')
print(f'[Dodge] if < {round(PetMissChance + DodgeChance,5)}')
print(f'[Parry] if < {round(PetMissChance + DodgeChance + ParryChance,5)}')
print(f'[Glancing] if < {round(PetMissChance + DodgeChance + ParryChance + GlancingChance,5)}')
print(f'[Block] if < {round(PetMissChance + DodgeChance + ParryChance + GlancingChance + PetBlockChance,5)}')
print(f'[Crit] if < {round(PetMissChance + DodgeChance + ParryChance + GlancingChance + PetBlockChance + PetCritChance,5)}')
print(f'else [Hit]')

HitChance = 0.06251524965208682
PetHitChance = 0.07
ArmorReductionFactor = 0.2589928057553957
MissChance = 0.017484750347913178
PetMissChance = 0.009999999999999995
BlockChance = 0.05
PetBlockChance = 0.05
CritChance = 0.07399637681159421
PetCritChance = 0.014000000000000005
CritEnhancementFactor = 2.3
GlancingChance = 0.4
GlancingReduction = 0.65
DodgeChance = 0.065
ParryChance = 0.14
[Miss] if < 0.01
[Dodge] if < 0.075
[Parry] if < 0.215
[Glancing] if < 0.615
[Block] if < 0.665
[Crit] if < 0.679
else [Hit]


In [17]:
def randomroller():
    global randomroll
    randomroll = round(np.random.randint(1,101) * 0.01, 2)
    return randomroll
    
def aoth_proc(improved_aoth, second):
    global aoth_modifier
    global aoth_ends_at
    a = randomroller()
    if a <= 0.03 * improved_aoth:
#         print(f'{round(second/100,2)}:[Imp. AotH] roll={a} en ioth = {0.03 * improved_aoth} en aoth duurt tot {(second + 1200)/100}')
        aoth_modifier = 0.03 * improved_aoth
        aoth_ends_at = second + 1200
        
def haste(aoth_ends_at, second, serpents_swiftness, gear_haste_rating, quiver_modifier):
    global HastedWeaponSpeed
    global aoth_modifier
    global haste_modifier
    if second >= aoth_ends_at:
        aoth_modifier = 0
        haste_modifier = (1+(serpents_swiftness*0.04)) * (1 + gear_haste_rating/15.77/100) * (1+quiver_modifier) * (1+aoth_modifier)
        HastedWeaponSpeed = WeaponSpeed / haste_modifier
#         return print(HastedWeaponSpeed)        
    else:    
        haste_modifier = (1+(serpents_swiftness*0.04)) * (1 + gear_haste_rating/15.77/100) * (1+quiver_modifier) * (1+aoth_modifier)
        HastedWeaponSpeed = WeaponSpeed / haste_modifier
#         return print(HastedWeaponSpeed)

def pet_haste(frenzy_ends_at, second, serpents_swiftness):
    global pet_HastedWeaponSpeed
    global frenzy_modifier
    global pet_haste_modifier
    if second >= frenzy_ends_at:
        frenzy_modifier = 0
        pet_haste_modifier = (1+(serpents_swiftness*0.04)) * (1+frenzy_modifier)
        PetHastedWeaponSpeed = 2 / pet_haste_modifier
#         return print(f'{second/100}: [PetHastedWeaponSpeed] = {PetHastedWeaponSpeed}')
    else:    
        pet_haste_modifier = (1+(serpents_swiftness*0.04)) * (1+frenzy_modifier)
        PetHastedWeaponSpeed = 2 / pet_haste_modifier
#         return print(f'{second/100}: [PetHastedWeaponSpeed] = {PetHastedWeaponSpeed}')

def frenzy_proc(frenzy, second):
    global frenzy_modifier
    global frenzy_ends_at
    a = randomroller()
    if a <= 0.2 * frenzy:
        print(f'{round(second/100,2)}: [Frenzy] duurt tot {(second + 800)/100}')
        frenzy_modifier = 0.3
        frenzy_ends_at = second + 800

In [82]:
# steady shot volgt 0.5/(1+haste) na de auto shot
second = 0
last_attack = ''
# uitzoeken hoe hitrating de tabel beinvloedt
            
def autoattackdef(second):
    global auto_attack_result
    global auto_attack_damage_final
    global number_of_autoattacks
    global totaldmg_autoattacks
    global last_autoattacked
    global last_attack
    global kill_command_start
    global kill_command_toggle
    
    randomroller()
    
    if randomroll <= MissChance:
        auto_attack_result = 'Miss'
        auto_attack_damage_final = 0
    
    elif mob_wears_shield and randomroll <= MissChance + BlockChance:
        auto_attack_result = 'Block'
        auto_attack_damage_final = 0
        
    elif mob_wears_shield and randomroll <= MissChance + BlockChance + CritChance:
        auto_attack_result = 'Crit'
        auto_attack_damage_final = CritEnhancementFactor * auto_attack_damage * (1-ArmorReductionFactor)
        if kill_command_start < second - 500:
#             print(f'heb gecrit -> ga nu kill command toggle aanzetten')
            kill_command_start = second
            kill_command_toggle = True
        
    elif randomroll < 1:
        auto_attack_result = 'Hit'
        auto_attack_damage_final = (1 + 0.01 * humanoid_monster_slaying) * auto_attack_damage * (1-ArmorReductionFactor)
    
    print(f'{second/100}: [Autoattack] {auto_attack_result} for {round(auto_attack_damage_final,2)} last_attack = {last_attack} last_autoattacked = {last_autoattacked}')    
    last_autoattacked = second
    number_of_autoattacks += 1
    last_attack = 'Auto Shot'
#     totaldmg_autoattacks += auto_attack_damage_final


def specialattackdef():
    global special_attack_result
    global special_attack_result_modifier
    global kill_command_start
    global kill_command_toggle    
    randomroller()
       
    if randomroll <= MissChance:
        special_attack_result = 'Miss'
        special_attack_result_modifier = 0
    elif mob_wears_shield and randomroll <= MissChance + BlockChance:
        special_attack_result = 'Block'
        special_attack_result_modifier = 0
    else:
        randomroller()
      
        if randomroll <= CritChance + kill_command_toggle * 0.1 * focused_fire:
            special_attack_result = 'Crit'
            special_attack_result_modifier = CritEnhancementFactor * (1-ArmorReductionFactor)
            frenzy_proc(frenzy, second)
            if kill_command_start < second - 500:
#                 print(f'heb gecrit -> ga nu kill command toggle aanzetten')
                kill_command_start = second
                kill_command_toggle = True
        else:
            special_attack_result = 'Hit'
            special_attack_result_modifier = (1 + 0.01 * humanoid_monster_slaying) * (1-ArmorReductionFactor)
            

def petautoattackdef():
    global pet_auto_attack_result
    global pet_auto_attack_damage_final
    global number_of_petautoattacks
    
    randomroller()
    
    if randomroll <= PetMissChance:
        pet_auto_attack_result = 'Miss'
        pet_auto_attack_damage_final = 0
        
    elif randomroll <= PetMissChance + DodgeChance:
        pet_auto_attack_result = 'Dodge'
        pet_auto_attack_damage_final = 0
        
    elif randomroll <= PetMissChance + DodgeChance + ParryChance:
        pet_auto_attack_result = 'Parry'
        pet_auto_attack_damage_final = 0
        
    elif randomroll <= PetMissChance + DodgeChance + ParryChance + GlancingChance:
        pet_auto_attack_result = 'Glancing Blow'
        pet_auto_attack_damage_final = GlancingReduction * auto_attack_damage * ArmorReductionFactor
        
    elif mob_wears_shield and randomroll <= PetMissChance + DodgeChance + ParryChance + GlancingChance + PetBlockChance:
        pet_auto_attack_result = 'Block'
        pet_auto_attack_damage_final = 0
        
    elif mob_wears_shield and randomroll <= PetMissChance + DodgeChance + ParryChance + GlancingChance + PetBlockChance + PetCritChance:
        pet_auto_attack_result = 'Crit'
        pet_auto_attack_damage_final = 2 * auto_attack_damage * ArmorReductionFactor
        frenzy_proc(frenzy, second)
        
    elif randomroll < 1:
        pet_auto_attack_result = 'Hit'
        pet_auto_attack_damage_final = auto_attack_damage * ArmorReductionFactor 
        
    number_of_petautoattacks += 1
    last_pet_attack = 'Auto Attack'
    print(f'{second/100}: [PetAutoattack] {pet_auto_attack_result} for {round(pet_auto_attack_damage_final,0)} based on roll of {randomroll}')
#     print(f'{second/100}: [Autoattack] {auto_attack_result} for {auto_attack_damage_final} last_attack = {last_attack} last_autoattacked = {last_autoattacked}')
    
def pet_specialattackdef():
    global pet_special_attack_result
    global pet_special_attack_result_modifier
    #First roll
    randomroller()
       
    if randomroll <= PetMissChance:
        pet_special_attack_result = 'Miss'
        pet_special_attack_result_modifier = 0
    elif randomroll <= PetMissChance + DodgeChance:
        pet_special_attack_result = 'Dodge'
        pet_special_attack_result_modifier = 0
    elif randomroll <= PetMissChance + DodgeChance + ParryChance:
        pet_special_attack_result = 'Parry'
        pet_special_attack_result_modifier = 0
    elif mob_wears_shield and randomroll <= PetMissChance + DodgeChance + ParryChance +  PetBlockChance:
        pet_special_attack_result = 'Block'
        pet_special_attack_result_modifier = 0
    else:
        randomroller()
        if randomroll <= PetCritChance:
            pet_special_attack_result = 'Crit'
            pet_special_attack_result_modifier = 2 * ArmorReductionFactor
        else:
            pet_special_attack_result = 'Hit'
            pet_special_attack_result_modifier = 1 * ArmorReductionFactor    
#     print(pet_special_attack_result)

In [53]:
def focus_regen(second): 
    global current_focus
    if second/100 % 4 == 0 and current_focus != 100:
        base_focus = 24.5 * (1 + 0.5 * bestial_discipline)
        if current_focus + base_focus >= 100:
            current_focus = max(100, current_focus)
        elif current_focus + base_focus < 100:
            current_focus += base_focus
        print(f'{second/100}: [Gained focus] {current_focus}')
        
        
def Ravage(second):
    global current_focus
    global last_ravage
    global pet_gcd_start    
    if second/100 - (1.5) > last_ravage/100 \
       and second/100 - (1.5) > pet_gcd_start/100 \
       and current_focus >= 25:
        randomroller()
        if randomroll > 0.5:
            ravagebonus = 2
        else:
            ravagebonus = 1
        pet_specialattackdef()
        ravage_damage = 0.5 * (50 + 70) *\
                        (1 + pet_happiness_bonus) *\
                        (1 + pet_family_bonus) *\
                        (1 + 0.04 * unleashed_fury) *\
                        pet_special_attack_result_modifier *\
                        ravagebonus
        current_focus -= 25        
        print(f'{second/100}: [Ravage] {pet_special_attack_result} for {round(ravage_damage,2)}. Current focus={current_focus}')
        last_ravage = second
        pet_gcd_start = second

        
        
def Bite(second):
    global current_focus
    global last_bite
    global pet_gcd_start
#     print(f'{second/100}: second = {second/100} and last_bite={last_bite/100} and current_focus={current_focus}')
    if second/100 > last_bite/100 + 10 \
       and second/100 - (1.5) > pet_gcd_start/100 \
       and current_focus >= 35:
        pet_specialattackdef()
        bite_damage = 0.5 * (108 + 132) *\
                        (1 + pet_happiness_bonus) *\
                        (1 + pet_family_bonus) *\
                        (1 + 0.04 * unleashed_fury) *\
                        pet_special_attack_result_modifier
        current_focus -= 35        
        print(f'{second/100}: [Bite] {pet_special_attack_result} for {round(bite_damage,2)}. Current focus={current_focus}')
        last_bite = second
        pet_gcd_start = second
        

In [76]:
# you want to start casting steady right after auto shot fire off.

def Steady_Shot(second,
                last_autoattacked,
                haste_modifier,
                human_latency,
                SteadyShotDamage):
    global last_attack
    global last_steady_shot
    global oom_to_use_steady_shot
    if (second - last_autoattacked - human_latency) > (1.5 * 100 / haste_modifier)\
        and last_autoattacked > 0 \
        and last_attack == 'Auto Shot' \
        and oom_to_use_steady_shot == False:
        specialattackdef()
        print(f'{second/100}: [Steady Shot] {special_attack_result} for {round(SteadyShotDamage,2)} based on roll of {randomroll} last_attack={last_attack}')
        last_attack = 'Steady Shot'
        last_steady_shot = second
        
def rapid_fire(aoth_ends_at, second, serpents_swiftness, gear_haste_rating, quiver_modifier):
    global HastedWeaponSpeed
    global haste_modifier
    global rapid_fire_toggle
    global rapid_fire_start
    if (second/100 >= rapid_fire_start/100 and second/100 < rapid_fire_start/100 + 15) or\
       (second/100 > rapid_fire_start/100 + (5-rapid_killing)*60 and rapid_fire_toggle == False):
        if rapid_fire_toggle == False:
            rapid_fire_toggle = True
            rapid_fire_start = second
#             print(f'{second/100}: [Rapid Fire] aangezet')
        haste_modifier = (1 + (serpents_swiftness * 0.04)) *\
                         (1 + gear_haste_rating / 15.77 / 100) *\
                         (1 + quiver_modifier) *\
                         (1 + aoth_modifier) *\
                         (1 + 0.4 * rapid_fire_toggle)
        HastedWeaponSpeed = WeaponSpeed / haste_modifier

#         return print(f'{second/100}: {HastedWeaponSpeed}')
    else:
#         if rapid_fire_toggle == True:
#             print(f'{second/100}: [Rapid Fire] uitgezet')        
        rapid_fire_toggle = False
        haste_modifier = (1 + (serpents_swiftness * 0.04)) *\
                         (1 + gear_haste_rating / 15.77 / 100) *\
                         (1 + quiver_modifier) *\
                         (1 + aoth_modifier) *\
                         (1 + 0.4 * rapid_fire_toggle)
        HastedWeaponSpeed = WeaponSpeed / haste_modifier
#         return print(f'{second/100}: [rapid fire] staat uit rapid_fire_toggle={rapid_fire_toggle}')
#         return print(f'{second/100}: {HastedWeaponSpeed}')


def KillCommand(second): #proct niet van snake trap, immolation trap. proct wel op melee autoattacks,raptor strike
    global kill_command_toggle
    global kill_command_start
#     print(f'{second/100} <= {kill_command_start/100 + 5}')
    if kill_command_toggle == True:
        specialattackdef()
        KillCommandDamage = ((60 + pet_ap_buffs) *\
                            (1 - cobra_reflexes * 0.16) + 127) *\
                            special_attack_result_modifier
        print(f'{second/100}: [Kill Command] {special_attack_result} for {round(KillCommandDamage,2)}.')
#         print(f'ga nu kill command toggle uitzetten')
        kill_command_toggle = False

In [81]:
iterations = 1
fight_duration = 500
aoth_ends_at = 0
frenzy_ends_at = 0
current_focus = 100
special_attack_result_modifier = 1
haste_modifier = (1+(serpents_swiftness*0.04)) * (1 + gear_haste_rating/15.77/100) * (1+quiver_modifier) * (1+aoth_modifier)
pet_haste_modifier = pet_haste_modifier = (1+(serpents_swiftness*0.04)) * (1+frenzy_modifier)
HastedWeaponSpeed = WeaponSpeed / haste_modifier
PetHastedWeaponSpeed = 2 / pet_haste_modifier
last_autoattacked = 0
last_steady_shot = 0
last_attack = ''
last_pet_attack = ''
last_ravage = -1.5
last_bite = - 10 * 100
pet_gcd_start = -1.5
kill_command_start = -5 * 100
kill_command_toggle = False
number_of_petautoattacks = 0
auto_attack_damage_final = 0
pet_auto_attack_damage_final = 0
oom_to_use_steady_shot = False
rapid_fire_toggle = False
rapid_fire_start = 0

for iteration in range(iterations):
    print(f'-------------------------------------------------------------------')
    for second in range(1, fight_duration*100 + 1):
#         print(f'{second/100}')
#         print(last_attack)
        haste(aoth_ends_at, second, serpents_swiftness, gear_haste_rating, quiver_modifier)
        pet_haste(frenzy_ends_at, second, serpents_swiftness)
        focus_regen(second)
        KillCommand(second)
        rapid_fire(aoth_ends_at, second, serpents_swiftness, gear_haste_rating, quiver_modifier)
        
        # auto attack
        if math.floor(100*(second/100%HastedWeaponSpeed))/100 == 0 and last_attack == '':
            autoattackdef(second)
            aoth_proc(improved_aoth, second)    
        elif math.floor(100*(second/100%HastedWeaponSpeed))/100 == 0 and oom_to_use_steady_shot == True:
            autoattackdef(second)
            aoth_proc(improved_aoth, second)
        elif last_attack == 'Steady Shot' and (second - last_steady_shot) > 0.5 * 100/haste_modifier:
#             print(f'second = {second} last_steady_shot={last_steady_shot} second - last_steady_shot = {second - last_steady_shot} > {0.5 * 100/haste_modifier}')
            autoattackdef(second)
            aoth_proc(improved_aoth, second)
        Steady_Shot(second,
                    last_autoattacked,
                    haste_modifier,
                    human_latency,
                    SteadyShotDamage)
        
        # pet auto attack
        if math.floor(100*(second/100%PetHastedWeaponSpeed))/100 == 0:
            petautoattackdef()
        Bite(second)
        Ravage(second)
        

-------------------------------------------------------------------
1.49: [Bite] Hit for 51.28. Current focus=65
1.56: [Autoattack] Hit for 403.79 last_attack =  last_autoattacked = 0
1.67: [PetAutoattack] Hit for 141.0 based on roll of 0.84
2.34: [Steady Shot] Hit for 747.77 based on roll of 0.46 last_attack=Auto Shot
2.6: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 156
2.99: [Ravage] Hit for 51.28. Current focus=40
3.28: [Steady Shot] Hit for 747.77 based on roll of 0.87 last_attack=Auto Shot
3.34: [PetAutoattack] Hit for 141.0 based on roll of 0.96
3.51: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 260
4.0: [Gained focus] 89.0
4.19: [Steady Shot] Hit for 747.77 based on roll of 0.56 last_attack=Auto Shot
4.42: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 351
4.5: [Ravage] Hit for 25.64. Current focus=64.0
5.01: [PetAutoattack] Glancing Blow for 92.0 based on roll of 0.24
5.1: [Steady Shot] Hit fo

170.5: [Steady Shot] Hit for 747.77 based on roll of 0.82 last_attack=Auto Shot
170.87: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 16941
171.67: [PetAutoattack] Glancing Blow for 92.0 based on roll of 0.23
171.82: [Steady Shot] Block for 747.77 based on roll of 0.04 last_attack=Auto Shot
172.0: [Gained focus] 57.0
172.0: [Ravage] Hit for 25.64. Current focus=32.0
172.14: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 17087
173.09: [Steady Shot] Block for 747.77 based on roll of 0.02 last_attack=Auto Shot
173.34: [PetAutoattack] Dodge for 0 based on roll of 0.06
173.41: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 17214
173.51: [Ravage] Hit for 51.28. Current focus=7.0
174.36: [Steady Shot] Hit for 747.77 based on roll of 0.81 last_attack=Auto Shot
174.68: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 17341
175.01: [PetAutoattack] Hit for 141.0 based on roll of 0.89
175.63:

343.39: [Steady Shot] Hit for 747.77 based on roll of 0.75 last_attack=Auto Shot
343.71: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 34244
344.0: [Gained focus] 49.0
344.0: [Ravage] Hit for 25.64. Current focus=24.0
344.66: [Steady Shot] Hit for 747.77 based on roll of 0.45 last_attack=Auto Shot
344.98: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 34371
345.01: [PetAutoattack] Parry for 0 based on roll of 0.13
345.93: [Steady Shot] Hit for 747.77 based on roll of 0.2 last_attack=Auto Shot
346.25: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 34498
346.67: [PetAutoattack] Hit for 141.0 based on roll of 0.93
347.34: [Steady Shot] Hit for 747.77 based on roll of 0.13 last_attack=Auto Shot
347.71: [Autoattack] Hit for 403.79 last_attack = Steady Shot last_autoattacked = 34625
348.0: [Gained focus] 73.0
348.0: [Bite] Hit for 51.28. Current focus=38.0
348.34: [PetAutoattack] Dodge for 0 based on roll of 0.

In [ ]:
a = 0.006086956521738962
math.floor(100*a)/100

In [ ]:
and(True,False)

In [ ]:
base_stats_df = pd.read_csv('base_stats.csv', index_col = 0)
base_stats_df['Melee Crit'] = round(base_stats_df['Melee Crit'] / 100,4)
base_stats_df.tail(5)

In [ ]:
#level x2
#strength x2
#agility x1
# mijn 31 druid heeft 202 ap

In [ ]:
mob_armor_health_df = pd.read_html('armor_health.html',
                                   skiprows = 6,
                                   index_col = 1)
mob_armor_health_df = mob_armor_health_df[0]

cols = ['Warrior','Paladin', 'Mage', 'Warrior.1', 'Paladin.1', 'Mage.1']
mob_armor_health_df = mob_armor_health_df.loc[:,cols]
mob_armor_health_df.columns = ['Warrior_Armor','Paladin_Armor', 'Mage_Armor', 'Warrior_Health', 'Paladin_Health', 'Mage_Health']
mob_armor_health_df.head()

In [ ]:

#untalented
# avg total damage=449.41824620053745
# avg total damage=453.5262784531543
# avg total damage=452.1229344815227
# avg total damage=450.8468846672712

#5/5 ferocity
# avg total damage=464.35763373853
# avg total damage=462.21241725761234
# avg total damage=460.3423069908101
# avg total damage=461.4671121242105

#5/5 natural weapons
# avg total damage=483.39410705848974
# avg total damage=481.9524527187383
# avg total damage=481.74963492069924
# avg total damage=482.9941847349497

#thorns can't crit

In [ ]:
# def Claw():
#     global ferocity_talentpoints
#     global number_of_claw
#     global totaldmg_claw
#     global current_energy
#     global number_of_claw_hits_and_crits
#     global ooc_proc_active
    
#     if (current_energy >= 45 - ferocity_talentpoints) or ooc_proc_active: 
#         if not ooc_proc_active:
#             current_energy -= (45 - ferocity_talentpoints)
#         elif ooc_proc_active:
#             ooc_proc_active = False
#             print(f' used up OOC proc on Claw and set to false')
#         specialattackdef()
#         print(f'Claw {special_attack_result} for {special_attack_result_modifier * claw_additional_damage} based on special_attack_result_modifier={special_attack_result_modifier} and base claw damage for {claw_additional_damage}')
#         number_of_claw += 1
#         OmenOfClarity()
#         totaldmg_claw += special_attack_result_modifier * claw_additional_damage
#         if special_attack_result == 'Crit' or special_attack_result == 'Hit':
#             ComboPoints()
# Claw()   

In [ ]:
# def OmenOfClarity():
#     global number_of_ooc_procs
#     global ooc_proc_active
    
#     if omen_of_clarity_talentpoints:
#         weapon_speed = 1
#         PPM = 2
#         proc_per_hit = weapon_speed * PPM / 60 
#         randomroller()
#         if randomroll <= proc_per_hit:
#             print(f'OOC procced, because randomroll={randomroll} and proc_per_hit={proc_per_hit}')
#             number_of_ooc_procs += 1
#             ooc_proc_active = True
        
# OmenOfClarity()

In [ ]:
def ComboPoints():
    global current_combopoints
    
    if current_combopoints < 5:
        combopoints_before = current_combopoints
        current_combopoints += 1
        combopoints_after = current_combopoints
        print(f'current_combopoints went from {combopoints_before} to {combopoints_after}')
    else:
        print(f'already at cap of {current_combopoints} combopoints')
ComboPoints()

In [ ]:
def Rip():
    global number_of_rips_cast
    global current_combopoints
    global totaldmg_rip
    global current_energy
    global starttime_rip
    global endtime_rip
    global fight_duration
    global second
    global current_rip_dmg_per_tick
    global ooc_proc_active
    global total_number_of_rip_ticks
    global current_rips_remaining
    
    if ((current_energy >= 30) or ooc_proc_active) and current_combopoints > 1 and endtime_rip !=fight_duration and (fight_duration-second)>1: 
        if not ooc_proc_active:
            current_energy -= 30
        elif ooc_proc_active:
            ooc_proc_active = False
            print(f' used up OOC proc on Rip and set to false')
        specialattackdef()
        number_of_rips_cast += 1
        print(f'{current_combopoints}cp Rip {special_attack_result} for {special_attack_result_modifier * rip_additional_damage} based on special_attack_result_modifier={special_attack_result_modifier} and base rip damage for {rip_additional_damage}')
        if special_attack_result == 'Hit' or special_attack_result == 'Crit':
        
            
            current_combopoints = 0

            starttime_rip = second
            endtime_rip = min(second + 12, fight_duration)
            current_rips_remaining = (endtime_rip - starttime_rip) // 2
            print(f'current_rips_remaining={current_rips_remaining} and total_number_of_rip_ticks voor={total_number_of_rip_ticks}')
            total_number_of_rip_ticks += current_rips_remaining
            print(f'total_number_of_rip_ticks na={total_number_of_rip_ticks}')

            current_rip_dmg_per_tick = special_attack_result_modifier * rip_additional_damage / 6
#             totaldmg_rip += current_rips_remaining * (special_attack_result_modifier * rip_additional_damage)

            print(f'rip starts at {starttime_rip} and ends at {endtime_rip}')
        OmenOfClarity()
        
Rip()   

In [ ]:
def RipTickDmg():
    global current_rip_dmg_per_tick
    global totaldmg_rip
    global second
    global starttime_rip
    global endtime_rip
    global current_rips_remaining
    
    if starttime_rip <= second <= endtime_rip:
        if (second - starttime_rip) % 2 == 0:
            print(f'we will have a rip tick for {current_rip_dmg_per_tick} because second={second} is in between {starttime_rip} and {endtime_rip}')
            totaldmg_rip += current_rip_dmg_per_tick
            current_rips_remaining -= 1

In [ ]:
#talents
natural_weapons_talentpoints = 5
natural_weapon_mult = 1 + 0.02 * natural_weapons_talentpoints
savage_fury_talentpoints = 0 #maul damage = (savage fury )*(natural weapons) *( base damage + maul bonus ) 
ferocity_talentpoints = 1
omen_of_clarity_talentpoints = 1
sharpened_claws_talentpoints = 0

current_energy = 0
current_level = 20
mob_level = 20
TargetLevel = mob_level
AttackerSkill = 5 * current_level #dit is een synoniem voor player attack rating
mob_defense_rating = 5 * mob_level
mob_wears_shield = True
NormalMissChance = 0.05
AttackerCrit = base_stats_df.at[current_level, 'Melee Crit'] + sharpened_claws_talentpoints * 0.02
mob_armor = mob_armor_health_df.at[mob_level, 'Warrior_Armor']
strength = base_stats_df.loc[current_level,'Strength']
agility = base_stats_df.loc[current_level,'Agility']
attack_power = current_level*2+2*strength+agility+ 90*(current_level/60)-20
auto_attack_damage_lowend = ((current_level*0.85) + (attack_power/14)) -10.1 *(current_level/60)
auto_attack_damage_highend = ((current_level*1.25) + (attack_power/14)) -10.1 *(current_level/60)
auto_attack_damage = (auto_attack_damage_lowend + auto_attack_damage_highend) /2
auto_attack_damage_final = 0.0
auto_attack_result = ''
randomroll = 0
print(f'auto_attack_damage = {auto_attack_damage}')
special_attack_result = ''
special_attack_result_modifier = 1
number_of_autoattacks = 0
totaldmg_autoattacks = 0.0
number_of_claw = 0
number_of_claw_hits_and_crits = 0
totaldmg_claw = 0.0
number_of_ooc_procs = 0
ooc_proc_active = False
total_number_of_rip_ticks = 0
number_of_rips_cast = 0
current_combopoints = 0
totaldmg_rip = 0
starttime_rip = 0
endtime_rip = 0
current_rip_dmg_per_tick = 0


#Claw rank damage bonus
if current_level < 28:
    claw_additional_damage = 27
elif current_level < 38:
    claw_additional_damage = 39
elif current_level < 48:
    claw_additional_damage = 57
elif current_level < 58:
    claw_additional_damage = 88
else:
    claw_additional_damage = 115
print(f'claw_additional_damage = {claw_additional_damage}')

#Rip rank damage bonus
if current_level < 28:
    rip_additional_damage = 42 + 0.24 * attack_power
elif current_level < 36:
    rip_additional_damage = 66 + 0.24 * attack_power
elif current_level < 44:
    rip_additional_damage = 90 + 0.24 * attack_power
elif current_level < 52:
    rip_additional_damage = 114 + 0.24 * attack_power
else:
    rip_additional_damage = 115 + 0.24 * attack_power
print(f'rip_additional_damage = {rip_additional_damage}')


# #Mob armor
# X = (0.1 * mob_armor) / (8.5 * current_level + 40)
# ArmorReductionFactor = 1 - (X / (1 + X))
# print(f'ArmorReductionFactor = {ArmorReductionFactor}')

# #Miss
# if TargetLevel < 10:
#     MissChance = NormalMissChance * (TargetLevel / 10)
# elif (mob_defense_rating - AttackerSkill) <= 10:
#     MissChance = 0.05 + (TargetLevel * 5 - AttackerSkill) * 0.001
# elif (mob_defense_rating - AttackerSkill) >= 11:
#     MissChance = 0.05 + (TargetLevel * 5 - AttackerSkill) * 0.002
# print(f'MissChance = {MissChance}')

# #Glancing
# # glancing gebeurt alleen op white non-crits
# GlancingChance = 0.10 + max(0, (TargetLevel - current_level) * 0.1)
# LowEndGlancing  = min(0.91, 1.3 - 0.05*(mob_defense_rating - AttackerSkill))
# HighEndGlancing = max(0.2, min(0.99, 1.2 - 0.03*(mob_defense_rating - AttackerSkill)))
# GlancingReduction =  max(0, (HighEndGlancing + LowEndGlancing) / 2)
# print(f'GlancingChance = {GlancingChance}')
# print(f'GlancingReduction = {GlancingReduction}')

# #Dodge
# DodgeChance = 0.05 + (TargetLevel*5 - AttackerSkill) * 0.001
# print(f'DodgeChance = {DodgeChance}')

# #Block
# if mob_wears_shield:
#     BlockChance = max(0, min(0.05, 0.05 + (TargetLevel*5 - AttackerSkill) * 0.001))
# else: BlockChance = 0
# print(f'BlockChance = {BlockChance}')

# #Crit
# if (AttackerSkill - mob_defense_rating) < 0:
#     CritChance = AttackerCrit + (AttackerSkill - mob_defense_rating) * 0.002
# elif (mob_defense_rating - AttackerSkill) >= 0:
#     CritChance = AttackerCrit + (AttackerSkill - mob_defense_rating) * 0.004
# print(f'CritChance = {CritChance}')

# #Parry
# if (mob_level - current_level) < 3:
#     ParryChance = 0.05 + 0.005 * (mob_level - current_level)
# elif (mob_level - current_level) >= 3:
#     ParryChance = 0.125 + 0.005 * (mob_level - current_level)
# print(f'ParryChance = {ParryChance}')

iterations = 1000
fight_duration = 18
for iteration in range(iterations):
    endtime_rip = 0
    current_combopoints = 0
    print(f'-------------------------------------------------------------------')
    for second in range(1, fight_duration + 1):
        energy_regen()
        autoattackdef()
        RipTickDmg()
        Rip()
        Claw()
        
print(f'avg number_of_autoattacks = {number_of_autoattacks/iterations}')
print(f'avg number_of_claw={number_of_claw/iterations}')
print(f'avg number_of_rips_cast={number_of_rips_cast/iterations}')
print(f'avg number_of_ooc_procs={number_of_ooc_procs/iterations}')
print(f'avg total_number_of_rip_ticks={total_number_of_rip_ticks/iterations}')    

print(f'avg totaldmg_autoattacks = {totaldmg_autoattacks/iterations}')
print(f'avg totaldmg_rip={totaldmg_rip/iterations}')
print(f'avg totaldmg_claw={totaldmg_claw/iterations}')
print(f'avg total damage={(totaldmg_autoattacks+totaldmg_claw+totaldmg_rip)/iterations}')

# print(f'current_rip_dmg_per_tick={current_rip_dmg_per_tick}')
# print(f'endtime_rip={endtime_rip}')
# print(f'starttime_rip={starttime_rip}')

level 20 test, 1000 iterations per keer, 20 seconden fight:  
https://classicdb.ch/?talent#0zLV0o  
avg total damage=495.84904525504555  
avg total damage=496.3233554543028  
avg total damage=490.1186092536928  
avg total damage=490.419443600198  
avg total damage=493.35048091867304  
avg total damage=491.521413205062 

https://classicdb.ch/?talent#0ZE00z  
avg total damage=469.60536778108207  
avg total damage=470.3019424180294  
avg total damage=468.67713137446356  
avg total damage=469.8835604524771  
avg total damage=471.0387953393395 

conclusie: op level 20 is het beter dat je natural weapons > ooc bent gegaan dan full feral

level 21 test (player en mob level ook 21 nu), 1000 iterations per keer, 20 seconden fight:  
https://classicdb.ch/?talent#0ZE00M  
avg total damage=500.80809050772757  
avg total damage=496.82136846348186  
avg total damage=498.5296860620977  
avg total damage=499.545095005327  
avg total damage=498.76305996801426  

https://classicdb.ch/?talent#0zLV0oZo  
avg total damage=511.86343686296675  
avg total damage=511.3034767323711  
avg total damage=512.6617021135  
avg total damage=513.6740494842375  
avg total damage=515.216063404812  

conclusie: op level 21 is het beter dat je natural weapons > ooc bent gegaan dan full feral

level 22 test (player en mob level ook 22 nu), 1000 iterations per keer, 20 seconden fight:  
https://classicdb.ch/?talent#0zLV0oZb  
avg total damage=539.4718155368879  
avg total damage=539.0400916631194  
avg total damage=539.6048825135542  
avg total damage=537.931288385405  
avg total damage=535.6836296996132  

https://classicdb.ch/?talent#0ZE00c  
avg total damage=529.1238945663637  
avg total damage=529.8980424644775  
avg total damage=529.3105151884452  
avg total damage=530.3454721678623  
avg total damage=530.6721833407895

conclusie: op level 22 is het beter dat je natural weapons > ooc bent gegaan dan full feral


https://www.youtube.com/watch?v=ivGeMsz_f4w&t=13279s 21 mob vs 21 mob duurt ongeveer 12-13 seconden voor Ebbnflow. Ga nu bij deze fight length proberen met level 20 of Rip waard is

https://classicdb.ch/?talent#0zLV0o met any cp rip > claw
avg total damage=333.65915839246344
avg total damage=329.7558811888153
avg total damage=331.25684170890946
avg total damage=332.7203872340704

https://classicdb.ch/?talent#0zLV0o met rip en ripticks uitgezet
avg total damage=317.6727283012795
avg total damage=317.529016379631
avg total damage=318.982360351264
avg total damage=319.2412588652787

https://classicdb.ch/?talent#0zLV0o met rip van minimaal 2cp > claw
avg total damage=318.389502634275
avg total damage=317.4986778791259
avg total damage=320.7427956771648
avg total damage=316.4483428571716
conclusie: bij een fight van maar 13 seconden is any cp rip beter dan geen rip, welke beter is dan 2cp rip

level 20, ooc enabled, fight duration 18 seconden. testen of 2cp rip al beter is dan geen rip
https://classicdb.ch/?talent#0zLV0o met any cp rip > claw
avg total damage=485.8089779129233
avg total damage=485.51953586631583
avg total damage=486.72844596425665

https://classicdb.ch/?talent#0zLV0o met rip en ripticks uitgezet
avg total damage=442.6683681189334
avg total damage=446.99619469779196
avg total damage=445.97107362383036

https://classicdb.ch/?talent#0zLV0o met rip van minimaal 2cp > claw
avg total damage=466.82674883490637
avg total damage=464.3862661263613
avg total damage=468.07971394804366
conclusie: bij een fight van 18 seconden is any cp rip beter dan 2cp rip, welke beter is dan geen rip

In [ ]:
# # Gruul's Lair:
# High King Maulgar: 7700
# Gruul the Dragonkiller: 7700

# Magtheridon's Lair:
# Magtheridon: 7700

# Zul'Aman:
# Nalorakk: 7700
# Akil'zon: 7700
# Jan'alai: 7700
# Halazzi: 7700
# Hex Lord Malacrass: 6200
# Zul'jin: 7700

# Serpentshrine Cavern:
# Hydross the Unstable: 7700
# The Lurker Below: 7700
# Leotheras the Blind: 7700
# Fathom-Lord Karathress 6200
# Morogrim Tidewalker: 7700
# Lady Vashj: 6200

# Tempest Keep:
# Void Reaver: 8800
# High Astromancer Solarian: 6200
# Al'ar: 7700
# Kael'thas Sunstrider: 6200

# Hyjal Summit:
# Rage Winterchill: 6200
# Anetheron: 6200
# Kaz'rogal: 6200
# Azgalor: 6200
# Archimonde: 6200

# Black Temple:
# High Warlord Naj'entus: 7700
# Supremus: 7700
# Shade of Akama: 7700
# Teron Gorefiend: 6200
# Gurtogg Bloodboil: 7700
# Reliquary of Souls:
# - Essence of Suffering: 0
# - Essence of Desire: 7700
# - Essence of Anger: 7700
# Mother Shahraz: 6200
# Illidari Council:
# - Gathios the Shatterer: 6200
# Illidan Stormrage: 7700

# Sunwell Plateau:
# Kalecgos (Dragon): 6200
# - Sathrovarr the Corruptor: 6200
# Brutallus: 7700
# Felmyst: 6200
# The Eredar Twins:
# - Grand Warlock Alythess: 6200
# - Lady Sacrolash: 6200
# M'uru: 7700
# - Shadowsword Fury Mage: 5700
# - Shadowsword Berserker: 7100
# - Void Sentinel: 6800

# Notice that all but 2 of the bosses have either 6200 or 7700 armor. A similar pattern occurs in the trash mobs: 5475/5700/5950 or 6800/7100/7400 for level 70/71/72 mobs. In terms of percentage reduction, these values result in either 34.15% or 39.15% armor reduction vs attackers of the same level as the mob.


# -- Karazhan

# -- Attumen the Huntsman: 41.5% -- 7500 7500
# -- Midnight: 43.5% -- 8200 7684
# UPDATE `creature_template` SET `armor`='7500' WHERE entry ='16152';
# UPDATE `creature_template` SET `armor`='8200' WHERE entry ='16151';

# --Maiden of Virtue: 39.0% -- 6700 6193
# UPDATE `creature_template` SET `armor`='6700' WHERE entry ='16457';

# -- Julianne: 36.5% -- 6200 6193
# -- Romulo: 42.0% -- 7700 7684
# -- Strawman: 41.0% -- 7700 7387
# -- Thinhead -- 10700 10387
# -- Roar: 31.0% -- 4700 7387
# -- Dorothee: -- 6200 6193
# -- Tito: -- 6700 6792
# -- The Crone: -- 6200 6193
# -- The Big Bad Wolf: -- 7700 7684
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='17534';
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='17533';
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='17543';
# UPDATE `creature_template` SET `armor`='10700' WHERE entry ='17547';
# UPDATE `creature_template` SET `armor`='4700' WHERE entry ='17546';
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='17535';
# UPDATE `creature_template` SET `armor`='6700' WHERE entry ='17548';
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='18168';
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='17521';

# -- The Curator: 36.5% -- 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='15691';

# -- Shade of Aran: 26.5% -- 3800 3878
# UPDATE `creature_template` SET `armor`='3800' WHERE entry ='17229';

# -- Kil'rek: 24.5% -- 3400 3427
# -- Terestian Illhoof: 39.0% -- 6700 6792
# UPDATE `creature_template` SET `armor`='3400' WHERE entry ='17229';
# UPDATE `creature_template` SET `armor`='6700' WHERE entry ='15688';

# -- Nightbane: 42.0% -- 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='17225';

# -- Netherspite: 34.0% -- 5500 5474
# UPDATE `creature_template` SET `armor`='5500' WHERE entry ='15689';

# -- Prince Malchezaar: 42.0% -- 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='15690';

# ---

# -- Gruul's Lair:

# -- High King Maulgar: 7700 7684
# -- Gicherer der Wahnsinnige: 6200 6193
# -- Blindauge der Seher: 6200 6193
# -- Olm der Beschwörer: 6200 6193
# -- Krosh Feuerhand: 6200 6193
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='18831';
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='18835';
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='18836';
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='18834';
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='18832';

# -- Gruul the Dragonkiller: 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='19044';

# ---

# -- Magtheridon's Lair:

# -- Magtheridon: 7700 7685
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='17257';

# ---

# -- Tempest Keep

# -- Al'ar: 40.0% -- 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='19514';

# -- Void Reaver: 50.0% -- 8800! 8806 // Sollte ja sonst auch Bleeds Immunity haben.
# UPDATE `creature_template` SET `armor`='8800' WHERE entry ='19516';

# -- High Astromancer Solarian: 37.0% -- 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='18805';

# -- Kael'thas Sunstrider: 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='19622';

# ---

# -- Serpentshrine Cavern:

# -- Hydross the Unstable: 7700 7685
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='21216';

# -- The Lurker Below: 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='21217';

# -- Leotheras the Blind: 7700 7685
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='21215';

# -- Fathom-Lord Karathress 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='21214';

# -- Morogrim Tidewalker: 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='21213';

# -- Lady Vashj: 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='21212';

# ---

# -- Zul'Aman:

# -- Nalorakk: 7700 7200
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='23576';
# -- Akil'zon: 7700 7700
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='23574';
# -- Jan'alai: 7700 0
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='23578';
# -- Halazzi: 7700 6900/7100
# UPDATE `creature_template` SET `armor`='7700' WHERE entry IN ('24144','23577');
# -- Hex Lord Malacrass: 6200 0
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='24239';
# -- Zul'jin: 7700 7700
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='23863';

# ---

# -- Hyjal Summit:

# -- Rage Winterchill: 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='17767';

# -- Anetheron: 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='17808';

# -- Kaz'rogal: 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='17888';

# -- Azgalor: 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='17842';

# -- Archimonde: 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='17968';

# ---

# -- Black Temple:

# -- High Warlord Naj'entus: 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='22887';

# -- Supremus: 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='22898';

# -- Shade of Akama: 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='22841';

# -- Teron Gorefiend: 6200 6194
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='22871';

# -- Gurtogg Bloodboil: 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='22948';

# -- Essence of Suffering: 0 0
# UPDATE `creature_template` SET `armor`='0' WHERE entry ='23418';

# -- Essence of Desire: 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='23419';

# -- Essence of Anger: 7700 7684
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='23420';

# -- Mother Shahraz: 6200 6193
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='22947';

# -- Illidari Council:
# -- Gathios the Shatterer: 6200 6193
# -- High Nethermancer Zerevor: 6200 6193
# -- Lady Malande: 6200 6193
# -- Veras Darkshadow: 200 7684
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='22949';
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='22950';
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='22951';
# UPDATE `creature_template` SET `armor`='200' WHERE entry ='22952';

# -- Illidan Stormrage: 7700 7685
# UPDATE `creature_template` SET `armor`='7700' WHERE entry ='22917';

# ---

# Sunwell Plateau:

# -- Kalecgos (Dragon): 6200 5500
# -- Sathrovarr the Corruptor: 6200 5500 
# -- Kalecgos (Human): 6200 5000 
# UPDATE `creature_template` SET `armor`='6200' WHERE entry IN ('24850','24892','24891');

# -- Brutallus: 7700 6200 
# UPDATE `creature_template` SET `armor`='7700' WHERE entry = '24882');

# -- Felmyst: 6200 5500 
# UPDATE `creature_template` SET `armor`='6200' WHERE entry = '25038');

# -- The Eredar Twins:
# -- Grand Warlock Alythess: 6200
# -- Lady Sacrolash: 6200
# UPDATE `creature_template` SET `armor`='6200' WHERE entry IN ('25165','25166');

# -- M'uru / Entropius: 7700 5200/5250
# UPDATE `creature_template` SET `armor`='7700' WHERE entry IN ('25741','25840');
# -- Shadowsword Fury Mage: 5700 4850
# UPDATE `creature_template` SET `armor`='5700' WHERE entry ='25799';

# -- Shadowsword Berserker: 7100 5150
# UPDATE `creature_template` SET `armor`='7100' WHERE entry ='25798';

# -- Void Sentinel: 6800 5000
# UPDATE `creature_template` SET `armor`='6800' WHERE entry ='25772';

# -- Kil'jaeden: 6200 5300
# UPDATE `creature_template` SET `armor`='6200' WHERE entry ='25315';

# -- Hand of the Deceiver: 5900 4800
# UPDATE `creature_template` SET `armor`='5900' WHERE entry ='25588';

# -- Sinister Reflection: 5475 ?
# UPDATE `creature_template` SET `armor`='5475' WHERE entry ='25708';